In [1]:
import torch                                                                    
import torch.nn as nn                                                           
import torch.nn.functional as F                                                 
import torch.optim as optim                                                     
import torchvision                                                              
from torchvision import datasets, transforms                                    
                                                                                
# parameters                                                                    
N_STEPS = 28                                                                    
N_INPUTS = 28                                                                   
N_NEURONS = 150                                                                 
N_OUTPUTS = 10                                                                  
N_EPHOCS = 10   

In [2]:
class ImageRNN(nn.Module):                                                      
    def __init__(self, batch_size, n_steps, n_inputs, n_neurons, n_outputs):    
        super(ImageRNN, self).__init__()                                        
                                                                                
        self.n_neurons = n_neurons                                              
        self.batch_size = batch_size                                            
        self.n_steps = n_steps                                                  
        self.n_inputs = n_inputs                                                
        self.n_outputs = n_outputs                                              
                                                                                
        self.basic_rnn = nn.RNN(self.n_inputs, self.n_neurons)                  
                                                                                
        self.FC = nn.Linear(self.n_neurons, self.n_outputs)                     
                                                                                
    def init_hidden(self,):                                                     
        # (num_layers, batch_size, n_neurons)                                   
        return (torch.zeros(1, self.batch_size, self.n_neurons))                
                                                                                
    def forward(self, X):                                                       
        # transforms X to dimensions: n_steps X batch_size X n_inputs           
        X = X.permute(1, 0, 2)                                                  
                                                                                
        self.batch_size = X.size(1)                                             
        self.hidden = self.init_hidden()                                        
                                                                                
        lstm_out, self.hidden = self.basic_rnn(X, self.hidden)                  
        out = self.FC(self.hidden)                                              
                                                                                
        return out.view(-1, self.n_outputs) # batch_size X n_output             
                                                                      

In [3]:
BATCH_SIZE = 64                                                                 
# list all transformations                                                      
transform = transforms.Compose( [transforms.ToTensor()])                        
                                                                                
data_dir="/home/mwang/Development/third-party/pytorch_examples/data"            
                                                                                
# download and load training dataset                                            
trainset = torchvision.datasets.MNIST(root=data_dir, train=True,                
                                        download=True, transform=transform)     
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,      
                                          shuffle=True, num_workers=2)          
                                                                                
# download and load testing dataset                                             
testset = torchvision.datasets.MNIST(root=data_dir, train=False,                
                                       download=True, transform=transform)      
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,        
                                         shuffle=False, num_workers=2)  

In [4]:
dataiter = iter(trainloader)  

In [5]:
images, labels = dataiter.next()

In [6]:
model = ImageRNN(BATCH_SIZE, N_STEPS, N_INPUTS, N_NEURONS, N_OUTPUTS)

In [10]:
images.shape

torch.Size([64, 1, 28, 28])

In [11]:
x = images.view(-1, 28,28)

In [13]:
x.shape

torch.Size([64, 28, 28])

In [14]:
 x2 = x.permute(1, 0, 2) 

In [15]:
x2.shape

torch.Size([28, 64, 28])

In [16]:
hidden = model.init_hidden()      

In [17]:
hidden

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

In [19]:
hidden.shape

torch.Size([1, 64, 150])

In [20]:
lstm_out, new_hidden = model.basic_rnn(x2, hidden)

In [22]:
lstm_out.shape

torch.Size([28, 64, 150])

In [23]:
new_hidden.shape

torch.Size([1, 64, 150])

In [24]:
lstm_out[27,:,:]

tensor([[-0.0747,  0.0250,  0.0103,  ...,  0.0546,  0.0179,  0.0498],
        [-0.0744,  0.0143,  0.0111,  ...,  0.0543,  0.0091,  0.0466],
        [-0.0720,  0.0224,  0.0250,  ...,  0.0657,  0.0397,  0.0355],
        ...,
        [-0.0832,  0.0448, -0.0002,  ...,  0.0517,  0.0317,  0.0520],
        [-0.0751,  0.0282,  0.0061,  ...,  0.0546,  0.0194,  0.0462],
        [-0.0783,  0.0204,  0.0111,  ...,  0.0554,  0.0194,  0.0386]],
       grad_fn=<SliceBackward>)

In [25]:
new_hidden

tensor([[[-0.0747,  0.0250,  0.0103,  ...,  0.0546,  0.0179,  0.0498],
         [-0.0744,  0.0143,  0.0111,  ...,  0.0543,  0.0091,  0.0466],
         [-0.0720,  0.0224,  0.0250,  ...,  0.0657,  0.0397,  0.0355],
         ...,
         [-0.0832,  0.0448, -0.0002,  ...,  0.0517,  0.0317,  0.0520],
         [-0.0751,  0.0282,  0.0061,  ...,  0.0546,  0.0194,  0.0462],
         [-0.0783,  0.0204,  0.0111,  ...,  0.0554,  0.0194,  0.0386]]],
       grad_fn=<StackBackward>)

In [26]:
lstm_out[26,:,:]

tensor([[-0.0762,  0.0291,  0.0066,  ...,  0.0549,  0.0240,  0.0466],
        [-0.0967,  0.0286,  0.0065,  ...,  0.0353,  0.0277,  0.0438],
        [-0.0340,  0.0224,  0.0250,  ...,  0.0863,  0.0169,  0.0467],
        ...,
        [-0.0692,  0.0385, -0.0044,  ...,  0.0702,  0.0049,  0.0599],
        [-0.0755,  0.0176,  0.0098,  ...,  0.0569,  0.0238,  0.0507],
        [-0.0785,  0.0200,  0.0113,  ...,  0.0566,  0.0205,  0.0373]],
       grad_fn=<SliceBackward>)

In [27]:
out=model.FC(new_hidden)

In [28]:
out.shape

torch.Size([1, 64, 10])

In [30]:
final_out=out.view(-1, 10)

In [31]:
final_out.shape

torch.Size([64, 10])

In [32]:
final_out

tensor([[-0.0312, -0.0886, -0.0473, -0.0334,  0.1358, -0.0124,  0.0517,  0.0333,
         -0.0302, -0.0768],
        [-0.0335, -0.0859, -0.0424, -0.0357,  0.1336, -0.0063,  0.0473,  0.0385,
         -0.0242, -0.0844],
        [-0.0477, -0.0841, -0.0437, -0.0107,  0.1447, -0.0239,  0.0525,  0.0312,
         -0.0394, -0.0840],
        [-0.0384, -0.0776, -0.0466, -0.0306,  0.1408, -0.0161,  0.0528,  0.0483,
         -0.0296, -0.0966],
        [-0.0334, -0.0843, -0.0455, -0.0327,  0.1407, -0.0164,  0.0487,  0.0351,
         -0.0292, -0.0794],
        [-0.0273, -0.1364, -0.0495, -0.0418,  0.1086, -0.0163,  0.0735,  0.0402,
         -0.0453, -0.0506],
        [-0.0325, -0.0828, -0.0422, -0.0332,  0.1371, -0.0074,  0.0493,  0.0280,
         -0.0269, -0.0719],
        [-0.0317, -0.0860, -0.0450, -0.0373,  0.1374, -0.0085,  0.0481,  0.0349,
         -0.0232, -0.0809],
        [-0.0314, -0.0580, -0.0402, -0.0477,  0.1180, -0.0138,  0.0357, -0.0024,
         -0.0333, -0.0638],
        [-0.0459, -

In [7]:
logits = model(images.view(-1, 28,28))

In [8]:
logits

tensor([[-0.0312, -0.0886, -0.0473, -0.0334,  0.1358, -0.0124,  0.0517,  0.0333,
         -0.0302, -0.0768],
        [-0.0335, -0.0859, -0.0424, -0.0357,  0.1336, -0.0063,  0.0473,  0.0385,
         -0.0242, -0.0844],
        [-0.0477, -0.0841, -0.0437, -0.0107,  0.1447, -0.0239,  0.0525,  0.0312,
         -0.0394, -0.0840],
        [-0.0384, -0.0776, -0.0466, -0.0306,  0.1408, -0.0161,  0.0528,  0.0483,
         -0.0296, -0.0966],
        [-0.0334, -0.0843, -0.0455, -0.0327,  0.1407, -0.0164,  0.0487,  0.0351,
         -0.0292, -0.0794],
        [-0.0273, -0.1364, -0.0495, -0.0418,  0.1086, -0.0163,  0.0735,  0.0402,
         -0.0453, -0.0506],
        [-0.0325, -0.0828, -0.0422, -0.0332,  0.1371, -0.0074,  0.0493,  0.0280,
         -0.0269, -0.0719],
        [-0.0317, -0.0860, -0.0450, -0.0373,  0.1374, -0.0085,  0.0481,  0.0349,
         -0.0232, -0.0809],
        [-0.0314, -0.0580, -0.0402, -0.0477,  0.1180, -0.0138,  0.0357, -0.0024,
         -0.0333, -0.0638],
        [-0.0459, -